In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/df2_final.csv")
df

,Unnamed: 0,Unnamed: 0.1,Composition,MaterialId,Reduced Formula,Elements,NSites,Volume,Density,Point Group,...,Crystal System,Corrected Energy,Formation Energy Per Atom,Decomposition Energy Per Atom,Dimensionality Cheon,Bandgap,Is Train,Decomposition Energy Per Atom All,Decomposition Energy Per Atom MP,Decomposition Energy Per Atom MP OQMD
0,0,0,209231,0,39871,163207,10,302.9202,3.2834,10,...,2,-70.4155,-1.9058,-0.1625,5,0.0023,1.0,0.0,-0.1625,-0.0733
1,1,1,355560,1,220471,181188,16,430.2851,6.8040,8,...,5,-94.0804,-0.5004,-0.0137,3,0.0022,1.0,0.0,-0.0137,-0.0106
2,2,2,173457,2,143223,83526,36,828.6647,12.7292,8,...,5,-237.6016,-1.2960,-0.1195,3,inf,1.0,0.0,-0.1195,-0.1185
3,3,3,344245,3,364467,102592,17,407.2614,8.6776,0,...,5,-93.6931,-0.3064,-0.0837,3,0.0011,1.0,-0.0,-0.0837,-0.0772
4,4,4,254904,4,78813,176755,36,727.5283,6.9851,8,...,5,-254.0953,-0.8718,-0.0186,3,inf,1.0,-0.0,-0.0186,-0.0176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384866,384866,384866,276307,384866,286928,176008,10,245.5848,8.2687,30,...,3,-48.0143,-0.5925,-0.0143,3,0.0009,1.0,0.0,-0.0143,-0.0138
384867,384867,384867,66679,384867,248601,46573,9,229.2117,8.8579,7,...,1,-33.6678,-0.6056,-0.0218,3,0.0198,1.0,-0.0,-0.0218,-0.0217
384868,384868,384868,104064,384868,14687,75275,38,391.5742,7.5986,13,...,6,-178.4183,-0.6005,-0.0252,5,2.5688,1.0,0.0,-0.0252,-0.0251
384869,384869,384869,36850,384869,133880,5174,22,876.4484,4.7103,26,...,1,-68.7495,-0.6063,-0.0189,5,inf,1.0,0.0,-0.0189,-0.0068


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384871 entries, 0 to 384870
Data columns (total 22 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Unnamed: 0                             384871 non-null  int64  
 1   Unnamed: 0.1                           384871 non-null  int64  
 2   Composition                            384871 non-null  int64  
 3   MaterialId                             384871 non-null  int64  
 4   Reduced Formula                        384871 non-null  int64  
 5   Elements                               384871 non-null  int64  
 6   NSites                                 384871 non-null  int64  
 7   Volume                                 384871 non-null  float64
 8   Density                                384871 non-null  float64
 9   Point Group                            384871 non-null  int64  
 10  Space Group                            384871 non-null  

In [4]:
df.shape

(384871, 22)

In [5]:
latent_dim = 100

In [6]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU,LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Reshape

In [7]:
generator = Sequential([
    Dense( 256,input_dim = latent_dim),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(1024),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(2048, activation="tanh"),
    LeakyReLU(),
    BatchNormalization()
])

In [8]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                       

In [9]:
degenerator = Sequential([
    Input(shape=(2048,1)),
    Dense(2048, activation=LeakyReLU(0.2)),
    Dense(1024, activation=LeakyReLU(0.2)),
    Dense(512, activation=LeakyReLU(0.2)),
    Dense(128, activation=LeakyReLU(0.2)),
    Dense(16, activation=LeakyReLU(0.2)),
    Dense(1, activation="sigmoid")
])

In [10]:
degenerator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2048, 2048)        4096      
                                                                 
 dense_5 (Dense)             (None, 2048, 1024)        2098176   
                                                                 
 dense_6 (Dense)             (None, 2048, 512)         524800    
                                                                 
 dense_7 (Dense)             (None, 2048, 128)         65664     
                                                                 
 dense_8 (Dense)             (None, 2048, 16)          2064      
                                                                 
 dense_9 (Dense)             (None, 2048, 1)           17        
                                                                 
Total params: 2694817 (10.28 MB)
Trainable params: 269

In [11]:
degenerator.trainable=False
gan_input = Input(shape=(latent_dim,))
gan_output = degenerator(generator(gan_input))
gan = Model(gan_input, gan_output)

degenerator.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.01, beta_1=0.5))
gan.compile(loss="mean_squared_error", optimizer = Adam(learning_rate=0.01, beta_1=0.5))

In [12]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 2048)              2797312   
                                                                 
 sequential_1 (Sequential)   (None, 2048, 1)           2694817   
                                                                 
Total params: 5492129 (20.95 MB)
Trainable params: 2789632 (10.64 MB)
Non-trainable params: 2702497 (10.31 MB)
_________________________________________________________________


In [13]:
latent_dim = 100
input_shape = (df.shape[1], 1)
batch_size = 2048
epochs = 100


In [ ]:
for epoch in range(epochs):
    # Generate real and fake samples
    idx = np.random.randint(0, 41881, size=batch_size)
    real_samples = compositions_scaled[idx]
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_samples = generator.predict(noise)
    fake_samples = np.mean(fake_samples, axis=1)
    fake_samples= np.expand_dims(fake_samples, axis=1)

    # Train the discriminator
    X = np.concatenate((real_samples, fake_samples), axis=0)
    y = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))), axis=0)

    discriminator_loss = degenerator.train_on_batch(X, y)

    # Train the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    y_gan = np.ones(batch_size)
    generator_loss = gan.train_on_batch(noise, y_gan)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

64/64 [==============================] - 1s 15ms/step


In [20]:
from sklearn.preprocessing  import MinMaxScaler
scaler = MinMaxScaler()
composition= df["Composition"].to_numpy().reshape(-1,1)
compositions_scaled = scaler.fit_transform(composition)

In [ ]:
num_materials = 10
noise = np.random.normal(0, 1, size=(num_materials, latent_dim))
new_compositions_encoded = generator.predict(noise)
new_compositions = scaler.inverse_transform(new_compositions_encoded)

# Print the generated materials
for composition in new_compositions:
    print(' '.join(composition[0].split()))